In [ ]:
%pip install pandas langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt langchain_upstage

In [2]:
import os
import pandas as pd
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

GPT_API_KEY = os.environ["GPT_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
UPSTAGE_API_KEY = os.environ["UPSTAGE_API_KEY"]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [ ]:
folder_path = '../data'

In [ ]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

In [3]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [4]:
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-index'

In [ ]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [5]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [ ]:
query = '결제 후 주문 취소가 가능한가요?'

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

In [ ]:
legacy_template = """
[context]: {context}
---
[질의]: {query}
위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답하세요.
단, [context] 정보에 없는 내용을 답해서는 안됩니다. 최대한 문장을 쉼표로 끊어서 대답하기 보다는, 온점으로 문장을 끊어주세요.
이 모든 정보를 종합해서 2~3줄의 구어체로 답해주세요.
"""

# LLM 질의 테스트

In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, 
"문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

    

In [17]:
# 챗봇에 질의
query = '박스가 파손되어 배송되었습니다.'

answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')

Answer : 죄송합니다.
 박스 파손으로 배송된 상품에 대해 불편을 드린 점 사과드립니다.
 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.
 파손 정도와 사진을 첨부하여 마이쿠팡을 통해 교환/반품 신청을 도와드리겠습니다.



**조건별 대응 방법:**

* **파손 정도가 심하고 상품에도 손상이 있는 경우:** 고객님께서 촬영하신 사진과 함께 주문번호를 확인하여 마이쿠팡 교환/반품 신청을 안내하고, 신청이 어려울 경우 고객센터 연결을 통해 직접 신청을 지원합니다.
 빠른 교환/반품 절차를 진행하여 불편을 최소화하도록 노력하겠습니다.


* **파손 정도가 경미하고 상품에는 문제가 없는 경우:** 고객님께서 사진을 보내주시면 파손 정도를 확인하고, 교환/반품이 아닌 부분적인 배송비 할인 등의 보상 방안을 제시합니다.
 (단, [context]에 명시된 교환/반품 기준에 따라 교환/반품이 불가능할 수 있습니다.
)

* **파손 정도 확인이 어려운 경우:** 고객님께 사진 첨부를 요청하고, 사진 확인 후 위 두 가지 경우 중 하나의 대응 방안을 선택하여 안내합니다.

.


In [23]:
retriever_chain = (
    retriever | merge_pages
)

ret = retriever_chain.invoke(query)
print(f"Answer: ", end='')
print(ret)

Answer: : 5
Question: Q[상품파손] 배송 받은 상품이 파손되었어요.
Answer: 상품이 파손되어 배송된 경우 정해진 기간 내에 교환 및 반품이 가능합니다. 마이쿠팡을 통해 교환 및 반품을 직접 신청해 주시거나 신청이 불가한 경우 고객센터로 연락 주시기 바랍니다. 
 교환/반품하기
마이쿠팡 → 주문목록 → 상품선택 → [교환 반품신청] 클릭
이후 각 단계에 해당하는 항목을 선택하여 신청을 완료합니다.
 교환/반품 안내사항 (클릭)
 파손에 의한 교환/반품 가능 기간 (클릭)
keywords: DELIVERY
count: 0
: 60
Question: Q[프레시백] 프레시백으로 주문했지만 박스나 비닐 포장으로 배송되는 경우도 있나요?
Answer: 네 수박 계란 등 프레시백에 담기 어려운 상품은 박스나 비닐로 포장되어 배송될 수 있습니다.
keywords: DELIVERY
count: 0
Question: 교환/반품 제한사항
Answer: 공통사항
• 주문제작 상품의 상품 제작이 이미 진행된 경우
• 고객의 사용 시간 경과 일부 소비에 의하여 상품의 가치가 현저히 감소한 경우
• 세트 상품 일부 사용 구성품을 분실하였거나 취급 부주의로 인한 파손/고장/오염으로 재판매 불가한 경우
• 모니터 해상도의 차이로 인해 색상이나 이미지가 실제와 다르다고 고객의 단순 변심으로 무료 교환/반품을 요청하는 경우
• 제조사의 사정(신모델 출시 부품가격 변동 등)에 의해 무료 교환/반품을 요청하는 경우
• 해외배송 상품이 세관에 걸려있을 경우 (상품 수령 후 가능)
• 해외배송 상품의 현지 사정이나 판매에 대한 가격 정책 등에 따라 제품의 가격 변동 사유로 인한 경우
 의류/잡화/수입명품/계절상품/식품/화장품
• 상품의 택(TAG) 및 라벨의 멸실 또는 훼손 상품의 사용 또는 훼손 구성품 누락으로 상품의 가치가 현저히 감소된 경우 
• 신선/냉장/냉동 상품의 단순 변심의 경우
• 뷰티 상품 이용 시 트러블(알레르기 붉은 반점 가려움 따가움 등)이 발생한 경우 진

In [25]:
print(ret)

: 5
Question: Q[상품파손] 배송 받은 상품이 파손되었어요.
Answer: 상품이 파손되어 배송된 경우 정해진 기간 내에 교환 및 반품이 가능합니다. 마이쿠팡을 통해 교환 및 반품을 직접 신청해 주시거나 신청이 불가한 경우 고객센터로 연락 주시기 바랍니다. 
 교환/반품하기
마이쿠팡 → 주문목록 → 상품선택 → [교환 반품신청] 클릭
이후 각 단계에 해당하는 항목을 선택하여 신청을 완료합니다.
 교환/반품 안내사항 (클릭)
 파손에 의한 교환/반품 가능 기간 (클릭)
keywords: DELIVERY
count: 0
: 60
Question: Q[프레시백] 프레시백으로 주문했지만 박스나 비닐 포장으로 배송되는 경우도 있나요?
Answer: 네 수박 계란 등 프레시백에 담기 어려운 상품은 박스나 비닐로 포장되어 배송될 수 있습니다.
keywords: DELIVERY
count: 0
Question: 교환/반품 제한사항
Answer: 공통사항
• 주문제작 상품의 상품 제작이 이미 진행된 경우
• 고객의 사용 시간 경과 일부 소비에 의하여 상품의 가치가 현저히 감소한 경우
• 세트 상품 일부 사용 구성품을 분실하였거나 취급 부주의로 인한 파손/고장/오염으로 재판매 불가한 경우
• 모니터 해상도의 차이로 인해 색상이나 이미지가 실제와 다르다고 고객의 단순 변심으로 무료 교환/반품을 요청하는 경우
• 제조사의 사정(신모델 출시 부품가격 변동 등)에 의해 무료 교환/반품을 요청하는 경우
• 해외배송 상품이 세관에 걸려있을 경우 (상품 수령 후 가능)
• 해외배송 상품의 현지 사정이나 판매에 대한 가격 정책 등에 따라 제품의 가격 변동 사유로 인한 경우
 의류/잡화/수입명품/계절상품/식품/화장품
• 상품의 택(TAG) 및 라벨의 멸실 또는 훼손 상품의 사용 또는 훼손 구성품 누락으로 상품의 가치가 현저히 감소된 경우 
• 신선/냉장/냉동 상품의 단순 변심의 경우
• 뷰티 상품 이용 시 트러블(알레르기 붉은 반점 가려움 따가움 등)이 발생한 경우 진료 확인서 및 

# OLLAMA TEST

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.
또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.

쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOllama(
    model="llama3.1:8b"
)

def merge_pages(pages):
    merged = "".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)
# 유사도 보기
# results = database.similarity_search_with_score(query=query, k=3)
# pprint(results)
query = '결제 후 주문 취소가 가능한가요?'
results = database.similarity_search_with_score(query=query, k=3)
pprint(results)

print()
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')
    